code 変更箇所の記載
- 引数
- メトリック記録

local 環境でのテスト

azure machine learning training

In [1]:
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset
from azureml.widgets import RunDetails

In [2]:
ws = Workspace.from_config()

In [3]:
dataset = Dataset.get_by_name(ws, name='cifar10')

In [4]:
experiment = Experiment(workspace=ws, name='day1-experiment-data')

config = ScriptRunConfig(source_directory='./code/pytorch-cloud',
                         script='train.py',
                         compute_target='cpu-cluster',
                         arguments=[
                            '--data_path', dataset.as_named_input('input').as_download(),
                            '--learning_rate', 0.003,
                            '--momentum', 0.92])



In [5]:
env = Environment.get(ws, "pytorch-env")

In [6]:
config.run_config.environment = env

In [7]:
# set up pytorch environment


run = experiment.submit(config)
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…